In [0]:
!pip install Faker

Requirement already satisfied: Faker in /databricks/python3/lib/python3.8/site-packages (13.12.1)
Requirement already satisfied: python-dateutil>=2.4 in /databricks/python3/lib/python3.8/site-packages (from Faker) (2.8.1)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.8/site-packages (from python-dateutil>=2.4->Faker) (1.15.0)
 WARNING: You are using pip version 21.0.1; however, version 22.1.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark import types as T
from snowflake.snowpark.types import *
from snowflake.snowpark.functions import udf, avg, col,lit,call_udf

User = dbutils.secrets.get("snowparkdetails", "username")
Password = dbutils.secrets.get("snowparkdetails", "password")
Account = dbutils.secrets.get("snowparkdetails", "account")
TenandId = dbutils.secrets.get("gen2-storage", "tenant-id")
SchemaName = "raw"
DatabaseName = "CustDB"
Warehouse = "cluster1"
DBrole = "ACCOUNTADMIN"
CONNECTION_PARAMETERS = {
    'account': Account,
    'user': User,
    'password': Password,
    'schema': SchemaName,
    'database': DatabaseName,
    'warehouse': Warehouse,
    'role':DBrole,
}

session = Session.builder.configs(CONNECTION_PARAMETERS).create()

print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

[Row(CURRENT_WAREHOUSE()='CLUSTER1', CURRENT_DATABASE()='NYCTAXI', CURRENT_SCHEMA()='TAXI')]

In [0]:
from faker import Faker

faker = Faker()


In [0]:
allempData = ((faker.name(), "Sales", 4000), \
    (faker.name(), "Sales", 5000),  \
    (faker.name(), "Sales", faker.random_int(5000, 10000)),   \
    (faker.name(), "Sales", 5000),    \
    (faker.name(), "Sales", faker.random_int(5000, 10000)), \
    (faker.name(), "Finance", 3000),  \
    (faker.name(), "Finance", 4500),  \
    (faker.name(), "Finance", 7500),    \
    (faker.name(), "IT", faker.random_int(5000, 10000)),\
    (faker.name(), "IT", faker.random_int(5000, 10000)),\
    (faker.name(), "Marketing", 5500), \
    (faker.name(), "Marketing", 5500),\
    (faker.name(), "Marketing", faker.random_int(5000, 10000))\
  )
 
columns= ["EmployeeName", "Department", "Salary"]
df = session.createDataFrame(data = allempData, schema = columns)
df.show()



-------------------------------------------------
"EMPLOYEENAME" |"DEPARTMENT" |"SALARY" |
-------------------------------------------------
Johnny Johnston |Sales |4000 |
Maurice Dean |Sales |5000 |
Devin Nash |Sales |6591 |
Veronica Davis |Sales |5000 |
Chelsea Schmidt |Sales |6647 |
Amanda Vargas |Finance |3000 |
Melissa Decker DDS |Finance |4500 |
Rodney Romero |Finance |7500 |
Jimmy Patterson PhD |IT |9798 |
Nathan Wilkinson |IT |7560 |
-------------------------------------------------

In [0]:
df.schema.fields

Out[51]: [StructField(EMPLOYEENAME, String, Nullable=False),
 StructField(DEPARTMENT, String, Nullable=False),
 StructField(SALARY, Long, Nullable=False)]

In [0]:
# from pyspark.sql.window import Window
# from pyspark.sql.functions import row_number,rank,dense_rank,percent_rank,ntile,lead,lag,col

from snowflake.snowpark import functions as F
from snowflake.snowpark.window import Window
from snowflake.snowpark.functions import row_number,rank,dense_rank,percent_rank,ntile,lead,lag,col,call_builtin,call_udf,avg,min,max,sum,array_agg
from snowflake.snowpark.types import IntegerType,StringType,DoubleType,StructField,StructType,FloatType

windowSpec  = Window.partition_by("Department").orderBy("Salary")
windowSpec_NOB  = Window.partition_by("Department")

df.withColumn("row_number",row_number().over(windowSpec)).toPandas()

Out[52]:

,EMPLOYEENAME,DEPARTMENT,SALARY,ROW_NUMBER
0,Nathan Wilkinson,IT,7560,1
1,Jimmy Patterson PhD,IT,9798,2
2,Travis Barker,Marketing,5500,1
3,Devin Ingram,Marketing,5500,2
4,Ashley Jones,Marketing,7382,3
5,Amanda Vargas,Finance,3000,1
6,Melissa Decker DDS,Finance,4500,2
7,Rodney Romero,Finance,7500,3
8,Johnny Johnston,Sales,4000,1
9,Maurice Dean,Sales,5000,2


In [0]:
df.withColumn("rank",rank().over(windowSpec)).\
withColumn("dense_rank",dense_rank().over(windowSpec)).filter(col("dense_rank")<=2).show()

-------------------------------------------------------------------------
"EMPLOYEENAME" |"DEPARTMENT" |"SALARY" |"RANK" |"DENSE_RANK" |
-------------------------------------------------------------------------
Amanda Vargas |Finance |3000 |1 |1 |
Melissa Decker DDS |Finance |4500 |2 |2 |
Nathan Wilkinson |IT |7560 |1 |1 |
Jimmy Patterson PhD |IT |9798 |2 |2 |
Travis Barker |Marketing |5500 |1 |1 |
Devin Ingram |Marketing |5500 |1 |1 |
Ashley Jones |Marketing |7382 |3 |2 |
Johnny Johnston |Sales |4000 |1 |1 |
Maurice Dean |Sales |5000 |2 |2 |
Veronica Davis |Sales |5000 |2 |2 |
-------------------------------------------------------------------------

In [0]:
df.withColumn("percentage_rank",percent_rank().over(windowSpec))\
.toPandas()

Out[54]:

,EMPLOYEENAME,DEPARTMENT,SALARY,PERCENTAGE_RANK
0,Nathan Wilkinson,IT,7560,0.00
1,Jimmy Patterson PhD,IT,9798,1.00
2,Travis Barker,Marketing,5500,0.00
3,Devin Ingram,Marketing,5500,0.00
4,Ashley Jones,Marketing,7382,1.00
5,Amanda Vargas,Finance,3000,0.00
6,Melissa Decker DDS,Finance,4500,0.50
7,Rodney Romero,Finance,7500,1.00
8,Johnny Johnston,Sales,4000,0.00
9,Maurice Dean,Sales,5000,0.25


In [0]:
df.withColumn("lag",lag("salary",1).over(windowSpec)).\
   withColumn("lead",lead("salary",1).over(windowSpec)) \
      .show()

------------------------------------------------------------------
"EMPLOYEENAME" |"DEPARTMENT" |"SALARY" |"LAG" |"LEAD" |
------------------------------------------------------------------
Johnny Johnston |Sales |4000 |NULL |5000 |
Maurice Dean |Sales |5000 |4000 |5000 |
Veronica Davis |Sales |5000 |5000 |6591 |
Devin Nash |Sales |6591 |5000 |6647 |
Chelsea Schmidt |Sales |6647 |6591 |NULL |
Nathan Wilkinson |IT |7560 |NULL |9798 |
Jimmy Patterson PhD |IT |9798 |7560 |NULL |
Travis Barker |Marketing |5500 |NULL |5500 |
Devin Ingram |Marketing |5500 |5500 |7382 |
Ashley Jones |Marketing |7382 |5500 |NULL |
------------------------------------------------------------------

In [0]:
df.withColumn("lead",lead("salary",1).over(windowSpec)) \
      .toPandas()

Out[56]:

,EMPLOYEENAME,DEPARTMENT,SALARY,LEAD
0,Nathan Wilkinson,IT,7560,9798.0
1,Jimmy Patterson PhD,IT,9798,NaN
2,Travis Barker,Marketing,5500,5500.0
3,Devin Ingram,Marketing,5500,7382.0
4,Ashley Jones,Marketing,7382,NaN
5,Amanda Vargas,Finance,3000,4500.0
6,Melissa Decker DDS,Finance,4500,7500.0
7,Rodney Romero,Finance,7500,NaN
8,Johnny Johnston,Sales,4000,5000.0
9,Maurice Dean,Sales,5000,5000.0


In [0]:
#Using built-in function in Snowflake - avg 
df.withColumn("AvgSal",call_builtin("avg", col("salary")).over(windowSpec_NOB)).withColumn("SumSal",sum(col("salary")).over(windowSpec_NOB)) \
   .toPandas()

Out[57]:

,EMPLOYEENAME,DEPARTMENT,SALARY,AVGSAL,SUMSAL
0,Johnny Johnston,Sales,4000,5447.600,27238
1,Maurice Dean,Sales,5000,5447.600,27238
2,Devin Nash,Sales,6591,5447.600,27238
3,Veronica Davis,Sales,5000,5447.600,27238
4,Chelsea Schmidt,Sales,6647,5447.600,27238
5,Amanda Vargas,Finance,3000,5000.000,15000
6,Melissa Decker DDS,Finance,4500,5000.000,15000
7,Rodney Romero,Finance,7500,5000.000,15000
8,Jimmy Patterson PhD,IT,9798,8679.000,17358
9,Nathan Wilkinson,IT,7560,8679.000,17358


In [0]:
#Withour orderby in window function
df.withColumn("AvgSal",avg("salary").over(windowSpec_NOB)).withColumn("SumSal",sum(col("salary")).over(windowSpec_NOB)) \
   .toPandas()

Out[58]:

,EMPLOYEENAME,DEPARTMENT,SALARY,AVGSAL,SUMSAL
0,Johnny Johnston,Sales,4000,5447.600,27238
1,Maurice Dean,Sales,5000,5447.600,27238
2,Devin Nash,Sales,6591,5447.600,27238
3,Veronica Davis,Sales,5000,5447.600,27238
4,Chelsea Schmidt,Sales,6647,5447.600,27238
5,Amanda Vargas,Finance,3000,5000.000,15000
6,Melissa Decker DDS,Finance,4500,5000.000,15000
7,Rodney Romero,Finance,7500,5000.000,15000
8,Jimmy Patterson PhD,IT,9798,8679.000,17358
9,Nathan Wilkinson,IT,7560,8679.000,17358


In [0]:
#with order by in window function
df.withColumn("AvgSal",avg("salary").over(windowSpec)).withColumn("SumSal",sum("salary").over(windowSpec)) \
    .show(12)

--------------------------------------------------------------------------
"EMPLOYEENAME" |"DEPARTMENT" |"SALARY" |"AVGSAL" |"SUMSAL" |
--------------------------------------------------------------------------
Johnny Johnston |Sales |4000 |4000.000000 |4000 |
Maurice Dean |Sales |5000 |4666.666667 |14000 |
Devin Nash |Sales |6591 |5147.750000 |20591 |
Veronica Davis |Sales |5000 |4666.666667 |14000 |
Chelsea Schmidt |Sales |6647 |5447.600000 |27238 |
Amanda Vargas |Finance |3000 |3000.000000 |3000 |
Melissa Decker DDS |Finance |4500 |3750.000000 |7500 |
Rodney Romero |Finance |7500 |5000.000000 |15000 |
Jimmy Patterson PhD |IT |9798 |8679.000000 |17358 |
Nathan Wilkinson |IT |7560 |7560.000000 |7560 |
Travis Barker |Marketing |5500 |5500.000000 |11000 |
Devin Ingram |Marketing |5500 |5500.000000 |11000 |
--------------------------------------------------------------------------

In [0]:

windowSpec_unboundedpre = Window.partitionBy(col("Department")).orderBy("Salary").rowsBetween(Window.unboundedPreceding,Window.currentRow)
windowSpec_prevrow = Window.partitionBy(col("Department")).orderBy("Salary").rowsBetween(-1,Window.currentRow)
windowSpec_nextvrow = Window.partitionBy(col("Department")).orderBy("Salary").rowsBetween(Window.currentRow,1)

In [0]:
#Running totals per department
windowSpec_unboundedpre = Window.partitionBy(col("Department")).orderBy("Salary")\
.rowsBetween(Window.unboundedPreceding,Window.currentRow)

df.withColumn("AvgSal",avg("Salary").over(windowSpec_unboundedpre))\
.withColumn("RunningTotal",sum(col("Salary")).over(windowSpec_unboundedpre)) \
    .show(11)

-----------------------------------------------------------------------------
"EMPLOYEENAME" |"DEPARTMENT" |"SALARY" |"AVGSAL" |"RUNNINGTOTAL" |
-----------------------------------------------------------------------------
Johnny Johnston |Sales |4000 |4000.000 |4000 |
Maurice Dean |Sales |5000 |4500.000 |9000 |
Veronica Davis |Sales |5000 |4666.666 |14000 |
Devin Nash |Sales |6591 |5147.750 |20591 |
Chelsea Schmidt |Sales |6647 |5447.600 |27238 |
Nathan Wilkinson |IT |7560 |7560.000 |7560 |
Jimmy Patterson PhD |IT |9798 |8679.000 |17358 |
Travis Barker |Marketing |5500 |5500.000 |5500 |
Devin Ingram |Marketing |5500 |5500.000 |11000 |
Ashley Jones |Marketing |7382 |6127.333 |18382 |
Amanda Vargas |Finance |3000 |3000.000 |3000 |
-----------------------------------------------------------------------------

In [0]:
# Sum with next rows
df.withColumn("SumSal",sum(col("Salary")).over(windowSpec_nextvrow)) \
    .show(11)

-----------------------------------------------------------
"EMPLOYEENAME" |"DEPARTMENT" |"SALARY" |"SUMSAL" |
-----------------------------------------------------------
Chelsea Schmidt |Sales |6647 |6647 |
Devin Nash |Sales |6591 |13238 |
Maurice Dean |Sales |5000 |11591 |
Veronica Davis |Sales |5000 |10000 |
Johnny Johnston |Sales |4000 |9000 |
Ashley Jones |Marketing |7382 |7382 |
Travis Barker |Marketing |5500 |12882 |
Devin Ingram |Marketing |5500 |11000 |
Rodney Romero |Finance |7500 |7500 |
Melissa Decker DDS |Finance |4500 |12000 |
Amanda Vargas |Finance |3000 |7500 |
-----------------------------------------------------------

In [0]:
#Sum with prev row
df.withColumn("SumSal",sum(col("Salary")).over(windowSpec_prevrow)) \
    .show(11)

-----------------------------------------------------------
"EMPLOYEENAME" |"DEPARTMENT" |"SALARY" |"SUMSAL" |
-----------------------------------------------------------
Johnny Johnston |Sales |4000 |4000 |
Maurice Dean |Sales |5000 |9000 |
Veronica Davis |Sales |5000 |10000 |
Devin Nash |Sales |6591 |11591 |
Chelsea Schmidt |Sales |6647 |13238 |
Travis Barker |Marketing |5500 |5500 |
Devin Ingram |Marketing |5500 |11000 |
Ashley Jones |Marketing |7382 |12882 |
Amanda Vargas |Finance |3000 |3000 |
Melissa Decker DDS |Finance |4500 |7500 |
Rodney Romero |Finance |7500 |12000 |
-----------------------------------------------------------

In [0]:
 #GroupBy and Agg 
df.groupBy(['Department']).agg([sum('Salary').alias("SumofSalary"), \
                                 avg('Salary').alias("AvgofSalary"), \
                                 max('Salary').alias("MaxofSalary")]).show()

----------------------------------------------------------------
"DEPARTMENT" |"SUMOFSALARY" |"AVGOFSALARY" |"MAXOFSALARY" |
----------------------------------------------------------------
Sales |27238 |5447.600000 |6647 |
Finance |15000 |5000.000000 |7500 |
IT |17358 |8679.000000 |9798 |
Marketing |18382 |6127.333333 |7382 |
----------------------------------------------------------------

In [0]:


newempData = ((faker.name(), "Sales", faker.random_int(5000, 10000)), \
    (faker.name(), "Sales", faker.random_int(5000, 10000)),  \
    (faker.name(), "Sales", faker.random_int(5000, 10000)),   \
    (faker.name(), "Sales", faker.random_int(5000, 10000)),    \
    (faker.name(), "Sales", faker.random_int(5000, 10000)), \
    (faker.name(), "Finance", faker.random_int(5000, 10000)),  \
    (faker.name(), "Finance", faker.random_int(5000, 10000)),  \
    (faker.name(), "Finance", faker.random_int(5000, 10000)),
    (faker.name(), "IT", faker.random_int(5000, 10000)),\
    (faker.name(), "IT", faker.random_int(5000, 10000)),\
    (faker.name(), "Marketing", faker.random_int(5000, 10000)), \
    (faker.name(), "Marketing", faker.random_int(5000, 10000)),\
    (None, "Marketing", faker.random_int(5000, 10000)),\
    (faker.name(), "Marketing", faker.random_int(5000, 10000))\
  )

empSchema = StructType([ \
    StructField("EmployeeName",StringType(),True), \
    StructField("Department",StringType(),True), \
    StructField("Salary", IntegerType(), True) \
  ])
 

df_udf = session.createDataFrame(data = newempData, schema = empSchema)


In [0]:
df_udf.toPandas()

Out[66]:

,EMPLOYEENAME,DEPARTMENT,SALARY
0,Tyler Miller,Sales,6443
1,Alicia Newton MD,Sales,9895
2,Mrs. Denise George,Sales,5535
3,Michael Mack,Sales,7439
4,Brian Davis,Sales,5577
5,Oscar Lowe,Finance,7029
6,Sarah Miller,Finance,9110
7,Patricia Wallace,Finance,7869
8,Jennifer Edwards,IT,8171
9,Heather Gray MD,IT,6537


In [0]:
type(df_udf)

Out[67]: snowflake.snowpark.dataframe.DataFrame

In [0]:
df_udf.createOrReplaceTempView("emp")

Out[71]: [Row(status='View EMP successfully created.')]

In [0]:
session.sql("select * from emp").show()

------------------------------------------------
"EMPLOYEENAME" |"DEPARTMENT" |"SALARY" |
------------------------------------------------
Tyler Miller |Sales |6443 |
Alicia Newton MD |Sales |9895 |
Mrs. Denise George |Sales |5535 |
Michael Mack |Sales |7439 |
Brian Davis |Sales |5577 |
Oscar Lowe |Finance |7029 |
Sarah Miller |Finance |9110 |
Patricia Wallace |Finance |7869 |
Jennifer Edwards |IT |8171 |
Heather Gray MD |IT |6537 |
------------------------------------------------

In [0]:
session.sql("list @azure_csv_stage/customer/parquetFiles/").show(max_width=1000)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
"name" |"size" |"md5" |"last_modified" |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/parquetFiles/part-00000-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-56-1-c000.snappy.parquet |1232348 |71c12069a63d65a0babd7d16808731f4 |Fri, 27 May 2022 08:49:01 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/parquetFiles/part-00001-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-57-1-c000.snappy.parquet |1232940 |3225ca79a5ccaf1591c17839fe13745c |Fri, 27 May 2022 08:48:57 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/parquetFiles/part-00002-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-58-1-c000.snappy.parquet |1234397 |8217ae32502b8b24bfdb65b22982acc9 |Fri, 27 May 2022 08:49:00 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/parquetFiles/part-00003-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-59-1-c000.snappy.parquet |1233382 |65865aadeafa03a5fc85963bc8df9184 |Fri, 27 May 2022 08:48:59 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/parquetFiles/part-00004-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-60-1-c000.snappy.parquet |1232687 |9ee710eb82a5cf48570cfb654c9d6c13 |Fri, 27 May 2022 08:48:58 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/parquetFiles/part-00005-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-61-1-c000.snappy.parquet |1235305 |f3b2986f21b313c02daf0b31180413cb |Fri, 27 May 2022 08:48:58 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/parquetFiles/part-00006-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-62-1-c000.snappy.parquet |1233292 |ca2bc1f0022066e72ee0e25c0d41a000 |Fri, 27 May 2022 08:48:58 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/parquetFiles/part-00007-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-63-1-c000.snappy.parquet |1233605 |0fe7b95a074c0305b53c036183568127 |Fri, 27 May 2022 08:48:59 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/parquetFiles/part-00008-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-64-1-c000.snappy.parquet |1231438 |61f8c3da7dfac0b51f2bb15304b03f9e |Fri, 27 May 2022 08:48:58 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/parquetFiles/part-00009-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-65-1-c000.snappy.parquet |1238351 |7aa9133e6867cad04e725bdc00826fb6 |Fri, 27 May 2022 08:48:59 GMT |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

##### Reading JSON Data

In [0]:
from snowflake.snowpark.functions import sql_expr,json_extract_path_text,parse_json,to_json,get_path

In [0]:
df_json = session.read.json("@azure_csv_stage/jsondata/")
df_json.select(sql_expr("$1[0]:Fuel").alias("Fuel")\
              ,sql_expr("$1[0]:Transmission").alias("Transmission")\
              ,sql_expr("$1[0]:about").alias("about")\
              ,sql_expr("$1[0]:cost").alias("Cost")).show()



--------------------------------------------------------------------------------------------------
"FUEL" |"TRANSMISSION" |"ABOUT" |"COST" |
--------------------------------------------------------------------------------------------------
"petrol" |"manual" |"Laborum Lorem duis eiusmod ex exercitation mol... |"$7,957.50" |
"petrol" |"manual" |"Ut ad non tempor incididunt irure in ea veniam... |"$5,208.73" |
"petrol" |"manual" |"Ipsum anim ullamco elit aliquip consequat volu... |"$9,625.96" |
"petrol" |"automatic" |"Proident anim ex adipisicing ut ut esse do qui... |"$2,797.75" |
"Electric" |"automatic" |"Labore officia in cupidatat eu commodo in Lore... |"$7,543.34" |
--------------------------------------------------------------------------------------------------

##### Reading CSV Files

In [0]:
cust_schema = StructType([
    StructField("C_CUSTKEY", IntegerType()),
    StructField("C_NAME", StringType()),
    StructField("C_ADDRESS", StringType()),
    StructField("C_NATIONKEY", ShortType()),
    StructField("C_PHONE", StringType()),
    StructField("C_ACCTBAL", DoubleType()),
    StructField("C_MKTSEGMENT", StringType()),
    StructField("C_COMMENT", StringType())
])

df_cust= session.read.option("skip_header", 1).option("field_delimiter", ",")\
.option("FIELD_OPTIONALLY_ENCLOSED_BY", '"')\
.schema(cust_schema).csv("@azure_csv_stage/customer/csvFiles/")
df_cust.show(3)

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
"C_CUSTKEY" |"C_NAME" |"C_ADDRESS" |"C_NATIONKEY" |"C_PHONE" |"C_ACCTBAL" |"C_MKTSEGMENT" |"C_COMMENT" |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
7673 |Customer#000007673 |nyMgsIXP0UOWJg8ro |0 |10-735-933-8462 |9541.65 |BUILDING |pinto beans cajole blithely regu |
19173 |Customer#000019173 |1lOQ6m4iVqkxMu,8azuFJeSyXY |0 |10-793-520-7121 |2232.36 |HOUSEHOLD |requests haggle quietly. silent frays detect. d... |
14844 |Customer#000014844 |yLIc6QRv4QT1S9LDTciIHBCQ6f |0 |10-141-789-2320 |8643.94 |BUILDING |above the blithely even excuses wake around the... |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

##### Reading and writing to Parquet

In [0]:
#Reading parquet Files
session.read.parquet("@azure_csv_stage/customer/parquetFiles/").select("C_NAME","C_PHONE").show(5)

----------------------------------------
"C_NAME" |"C_PHONE" |
----------------------------------------
Customer#000005500 |10-931-168-1559 |
Customer#000021746 |10-961-333-3043 |
Customer#000017347 |10-987-905-6656 |
Customer#000014783 |10-937-220-7748 |
Customer#000003815 |10-968-870-7672 |
----------------------------------------

In [0]:
#Writing to Parquet without any partitions
df_cust.write.copy_into_location('@azure_csv_stage/customer/newsnowparkParquetFiles/', \
 file_format_type="parquet", header=True,OVERWRITE=True)

Out[103]: [Row(rows_unloaded=150000, input_bytes=13981476, output_bytes=13981476)]

In [0]:
#Writing to Parquet with partitions
df_cust.write.copy_into_location('@azure_csv_stage/customer/newsnowparkpartitionedParquetFiles/', 
 partition_by="'C_MKTSEGMENT=' || C_MKTSEGMENT || '/C_NATIONKEY=' || to_varchar(C_NATIONKEY)" ,\
 file_format_type="parquet", header=True)

Out[106]: [Row(rows_unloaded=150000, input_bytes=13892771, output_bytes=13892771)]

In [0]:

session.sql("list @azure_csv_stage/customer/newsnowparkpartitionedParquetFiles").show(max_width=1000)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
"name" |"size" |"md5" |"last_modified" |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/newsnowparkpartitionedParquetFiles |0 |NULL |Wed, 8 Jun 2022 04:35:47 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/newsnowparkpartitionedParquetFiles/C_MKTSEGMENT=AUTOMOBILE |0 |NULL |Wed, 8 Jun 2022 04:35:47 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/newsnowparkpartitionedParquetFiles/C_MKTSEGMENT=AUTOMOBILE/C_NATIONKEY=0 |0 |NULL |Wed, 8 Jun 2022 04:35:47 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/newsnowparkpartitionedParquetFiles/C_MKTSEGMENT=AUTOMOBILE/C_NATIONKEY=0/data_01a4cd73-0000-f62d-0000-cd4d005e82a6_106_4_0.snappy.parquet |2754948 |c0965e707a18fd87830a82e2e7a000b7 |Wed, 8 Jun 2022 04:35:47 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/newsnowparkpartitionedParquetFiles/C_MKTSEGMENT=AUTOMOBILE/C_NATIONKEY=0/data_01a4d2bc-0000-f62d-0000-cd4d005ec066_106_4_0.snappy.parquet |2754948 |c0965e707a18fd87830a82e2e7a000b7 |Thu, 9 Jun 2022 03:08:50 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/newsnowparkpartitionedParquetFiles/C_MKTSEGMENT=AUTOMOBILE/C_NATIONKEY=0/data_01a4d2be-0000-f697-0000-cd4d005eb1c2_106_4_0.snappy.parquet |2754948 |c0965e707a18fd87830a82e2e7a000b7 |Thu, 9 Jun 2022 03:10:17 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/newsnowparkpartitionedParquetFiles/C_MKTSEGMENT=BUILDING |0 |NULL |Wed, 8 Jun 2022 04:35:47 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/newsnowparkpartitionedParquetFiles/C_MKTSEGMENT=BUILDING/C_NATIONKEY=0 |0 |NULL |Wed, 8 Jun 2022 04:35:47 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/newsnowparkpartitionedParquetFiles/C_MKTSEGMENT=BUILDING/C_NATIONKEY=0/data_01a4cd73-0000-f62d-0000-cd4d005e82a6_006_5_0.snappy.parquet |2793045 |428fd6aa7fa0c983dd688283c24e3c1d |Wed, 8 Jun 2022 04:35:47 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/newsnowparkpartitionedParquetFiles/C_MKTSEGMENT=BUILDING/C_NATIONKEY=0/data_01a4d2bc-0000-f62d-0000-cd4d005ec066_006_5_0.snappy.parquet |2793045 |428fd6aa7fa0c983dd688283c24e3c1d |Thu, 9 Jun 2022 03:08:50 GMT |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

##### Writing to Table

In [0]:
df_cust.write.mode("append").saveAsTable("NewCustomers")

In [0]:
session.sql("list @azure_csv_stage/customer/csvFiles/").show(max_width=100)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
"name" |"size" |"md5" |"last_modified" |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00000-tid-320033463... |2423786 |4532b55261a23f64df4ab6c62eb40895 |Fri, 27 May 2022 08:48:15 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00001-tid-320033463... |2423268 |48786b81aadb83782db0b060917e783e |Fri, 27 May 2022 08:48:16 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00002-tid-320033463... |2427801 |a34cebd103a224413f021cc837c430cd |Fri, 27 May 2022 08:48:15 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00003-tid-320033463... |2419248 |5870449c4aba96cb5ac1be58f2473dce |Fri, 27 May 2022 08:48:19 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00004-tid-320033463... |2425732 |bb4543913b6d0440c4e86787abd7cbfd |Fri, 27 May 2022 08:48:21 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00005-tid-320033463... |2419015 |458109f9c44dc6a66e7314497a61047d |Fri, 27 May 2022 08:48:20 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00006-tid-320033463... |2422780 |eb27a0f1d757a4c9fc95a401806941af |Fri, 27 May 2022 08:48:17 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00007-tid-320033463... |2421433 |cd00b545e19f8e13a7261e3773e81575 |Fri, 27 May 2022 08:48:19 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00008-tid-320033463... |2421195 |672ac836176a52223f38db9473b30088 |Fri, 27 May 2022 08:48:25 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00009-tid-320033463... |2422384 |c3b18626b798eab0e955c591ad8f9c4e |Fri, 27 May 2022 08:48:19 GMT |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
df_stages=session.sql(" list @azure_csv_stage/sesnsordata/").collect()
for val in df_stages:
    print(val.name)

azure://[REDACTED]demostgacct.blob.core.windows.net/staging/sesnsordata/Vehicle1.json
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/sesnsordata/Vehicle2.json
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/sesnsordata/Vehicle3.json
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/sesnsordata/Vehicle4.json
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/sesnsordata/Vehicle5.json